In [1]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression, LassoCV
from sklearn.model_selection import GroupKFold
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import r2_score, mean_squared_error

import warnings
warnings.filterwarnings('ignore')

In [2]:
data = pd.read_csv("C:/Users/hongz/Downloads/choices13k-main/reg_data_main2.csv")
# Modify graph_id as specified
data['graph_id'] = data['graph_id'] + 100 * data['reshuffle_ind']
data.head()

,subject_id,graph_id,time,click,risk,forecast,confidence,investment,reason_risky,reason_confidence,...,gender_ind,employment_ind,education_ind,income_ind,statistics_ind,risk-taking_ind,stock knowledge_ind,frequency _ind,history_ind,technical_ind
0,5fbb4426e47b46c3e2eeb544,156,11.550,6,6,112,62,59,NaN,NaN,...,0,0,1,1,0,0,0,0,0,0
1,65981b2c1df3be0020afa351,154,57.601,10,8,95,14,9,The ones that had big dips in them.,A lot of them. Only because I'm slowly trying ...,...,0,1,0,0,1,1,0,1,0,0
2,655791684bb1c5db02826d17,192,48.958,16,7,91,100,51,If it fluctuates more than 20% within 12 months.,With a $3 investment I do not feel unconfident...,...,0,1,1,1,0,1,0,1,1,0
3,62ddbd7eb3e9431e49b46ec1,182,49.950,11,8,115,79,60,NaN,NaN,...,1,1,1,0,1,1,0,0,0,0
4,5fb13091b87dfd5888f73e05,180,45.070,8,5,93,20,0,Those that have big drops.,When they are inconsistent and up and down on ...,...,0,0,0,0,0,1,0,1,1,0


In [3]:
from sklearn.preprocessing import LabelEncoder

# Initialize the LabelEncoder
le = LabelEncoder()

# Fit and transform 'subject_id' to numeric
data['subject_id_encoded'] = le.fit_transform(data['subject_id'])

data.head()

,subject_id,graph_id,time,click,risk,forecast,confidence,investment,reason_risky,reason_confidence,...,employment_ind,education_ind,income_ind,statistics_ind,risk-taking_ind,stock knowledge_ind,frequency _ind,history_ind,technical_ind,subject_id_encoded
0,5fbb4426e47b46c3e2eeb544,156,11.550,6,6,112,62,59,NaN,NaN,...,0,1,1,0,0,0,0,0,0,185
1,65981b2c1df3be0020afa351,154,57.601,10,8,95,14,9,The ones that had big dips in them.,A lot of them. Only because I'm slowly trying ...,...,1,0,0,1,1,0,1,0,0,461
2,655791684bb1c5db02826d17,192,48.958,16,7,91,100,51,If it fluctuates more than 20% within 12 months.,With a $3 investment I do not feel unconfident...,...,1,1,1,0,1,0,1,1,0,438
3,62ddbd7eb3e9431e49b46ec1,182,49.950,11,8,115,79,60,NaN,NaN,...,1,1,0,1,1,0,0,0,0,307
4,5fb13091b87dfd5888f73e05,180,45.070,8,5,93,20,0,Those that have big drops.,When they are inconsistent and up and down on ...,...,0,0,0,0,1,0,1,1,0,183


In [4]:
# Define independent variables for the baseline regression
X_base = data[['Obj_Std', 'Skewness', 'Kurtosis', 'time', 'click', 'order']]

# Split the data
X_train_base, X_test_base, y_train, y_test = train_test_split(X_base, data['risk'], test_size=0.2, random_state=42)

# Baseline regression
model_base = LinearRegression()
model_base.fit(X_train_base, y_train)
y_pred_base = model_base.predict(X_test_base)
r2_base = r2_score(y_test, y_pred_base)
r2_base

0.034867231019441225

In [5]:
X_3_feature = data[['Obj_Std', 'Skewness', 'Kurtosis', 'time', 'click', 'order',  'Recency_Factor', 'Rep_Factor', 'Sign_Factor']]
X_train_3, X_test_3 = train_test_split(X_3_feature, test_size=0.2, random_state=42)

model_3_feature = LinearRegression()
model_3_feature.fit(X_train_3, y_train)
y_pred_3 = model_3_feature.predict(X_test_3)
r2_3_feature = r2_score(y_test, y_pred_3)
r2_3_feature

0.10903025158853219

In [6]:
X_8_feature = data[['Obj_Std', 'Skewness', 'Kurtosis', 'time', 'click', 'order',  'Recency_Factor', 'Rep_Factor', 'Sign_Factor', 'SH_Rep_Factor', 'SH_Sign_Factor', 'SH_Obj_Std', 'SH_Skewness', 'SH_Kurtosis']]
X_train_8, X_test_8 = train_test_split(X_8_feature, test_size=0.2, random_state=42)

model_8_feature = LinearRegression()
model_8_feature.fit(X_train_8, y_train)
y_pred_8 = model_8_feature.predict(X_test_8)
r2_8_feature = r2_score(y_test, y_pred_8)
r2_8_feature

0.11420915583952151

In [7]:
graph_id_dummies = pd.get_dummies(data['graph_id'], prefix='graph_id')
data_with_fe = pd.concat([data, graph_id_dummies], axis=1)

# Assuming 'data' is your DataFrame and 'graph_id' is the categorical variable
graph_id_dummies = pd.get_dummies(data['graph_id'], prefix='graph_id')
data_with_fe = pd.concat([data, graph_id_dummies], axis=1)

# Create a list of features including the dummy variables
features = ['time', 'click', 'order'] + list(graph_id_dummies.columns)

# Select these features for X
X_graph_id_fe = data_with_fe[features]

# Assuming y_train and y_test are defined or create them if not mentioned
X_train_graph_id_fe, X_test_graph_id_fe = train_test_split(X_graph_id_fe, test_size=0.2, random_state=42)
#X_train_graph_id_fe, X_test_graph_id_fe, y_train, y_test = train_test_split(X_graph_id_fe, data['confidence'], test_size=0.2, random_state=42)

# Fit the baseline regression model
model_graph_id_fe = LinearRegression()
model_graph_id_fe.fit(X_train_graph_id_fe, y_train)

# Predict and calculate the R-squared value
y_pred_graph_id_fe = model_graph_id_fe.predict(X_test_graph_id_fe)
r2_graph_id_fe = r2_score(y_test, y_pred_graph_id_fe)

r2_graph_id_fe

0.16971902293361563

With subject_id_encoded Fixed Effect

In [8]:
# Create dummy variables for 'subject_id_encoded' and 'graph_id'
subject_id_dummies = pd.get_dummies(data['subject_id_encoded'], prefix='subject_id')
#graph_id_dummies = pd.get_dummies(data['graph_id'], prefix='graph_id')

# Add these dummy variables to the data
data_with_dummies = pd.concat([data, subject_id_dummies], axis=1)
data_with_dummies = pd.concat([data_with_dummies, graph_id_dummies], axis=1)
# Define the baseline features including the subject_id dummies
baseline_features_with_dummies = ['Obj_Std', 'Skewness', 'Kurtosis', 'time', 'click', 'order'] + list(subject_id_dummies.columns)
#baseline_features_with_dummies

In [9]:
# Define the independent and dependent variables
X_baseline_with_dummies = data_with_dummies[baseline_features_with_dummies]
y = data_with_dummies['risk']

# Split the data into training and testing sets
X_train_baseline, X_test_baseline, y_train, y_test = train_test_split(X_baseline_with_dummies, y, test_size=0.2, random_state=42)

# Fit the baseline regression model
model_baseline_with_dummies = LinearRegression()
model_baseline_with_dummies.fit(X_train_baseline, y_train)

# Predict and calculate the R-squared value
y_pred_baseline = model_baseline_with_dummies.predict(X_test_baseline)
r2_baseline_fe = r2_score(y_test, y_pred_baseline)

r2_baseline_fe

0.160245352228657

Lasso

In [10]:
def lasso_kfold_cv(data, features, target):
    X = data[features]
    y = data[target]

    # Standardizing the data
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    # Setting up 10-fold cross-validation
    kf = KFold(n_splits=10, shuffle=True, random_state=42)
    
    # Using LassoCV to find the best alpha during cross-validation
    lasso_cv = LassoCV(alphas=np.logspace(-6, 0, 100), cv=kf, random_state=42)
    lasso_cv.fit(X_scaled, y)

    # Evaluating the model using cross-validation
    cv_scores = cross_val_score(lasso_cv, X_scaled, y, cv=kf, scoring='r2')
    mean_r2 = np.mean(cv_scores)
    std_r2 = np.std(cv_scores)

    return mean_r2, std_r2

# Using the model
features_base = ['Obj_Std', 'Skewness', 'Kurtosis', 'time', 'click', 'order', 'subject_id_encoded', 'Recency_Factor', 'Rep_Factor', 'Sign_Factor', 'SH_Rep_Factor', 'SH_Sign_Factor', 'SH_Obj_Std', 'SH_Skewness', 'SH_Kurtosis']
r2_8_feature_lasso, std_8_feature_lasso = lasso_kfold_cv(data, features_base, 'risk')
print(f"Mean R-squared for Lasso with 10-Fold CV: {r2_8_feature_lasso:.4f} ± {std_8_feature_lasso:.4f}")

Mean R-squared for Lasso with 10-Fold CV: 0.1180 ± 0.0092


In [11]:
# Extracting ret and price features
ret_features = [col for col in data.columns if col.startswith('ret_') and 'diff' not in col]
price_features = [col for col in data.columns if col.startswith('price_') and 'diff' not in col]

# Large Lasso features include base features plus all ret_ and price_ values
features_large = features_base + ret_features + price_features

# Adding first and second differences for Huge Lasso features
ret_diff_features = [col for col in data.columns if 'ret_' in col and 'diff' in col]
price_diff_features = [col for col in data.columns if 'price_' in col and 'diff' in col]

features_huge = features_large + ret_diff_features + price_diff_features

# Calculate R-squared for Large Lasso
r2_large_lasso, std_r2_large = lasso_kfold_cv(data, features_large, 'risk')
print(f"Mean R-squared for Large Lasso with 10-Fold CV: {r2_large_lasso:.4f} ± {std_r2_large:.4f}")

# Calculate R-squared for Huge Lasso
r2_huge_lasso, std_r2_huge = lasso_kfold_cv(data, features_huge, 'risk')
print(f"Mean R-squared for Huge Lasso with 10-Fold CV: {r2_huge_lasso:.4f} ± {std_r2_huge:.4f}")

Mean R-squared for Large Lasso with 10-Fold CV: 0.1525 ± 0.0085
Mean R-squared for Huge Lasso with 10-Fold CV: 0.1522 ± 0.0083


lasso within subjected_id

In [12]:
from sklearn import set_config
set_config(enable_metadata_routing=False)  # Explicitly disable metadata routing

def lasso_group_kfold_cv(data, features, target):
    X = data[features]
    y = data[target]
    groups = data['subject_id_encoded']  # This assumes subject_id_encoded is a column in the DataFrame

    # Setting up 10-fold cross-validation that respects groups
    gkf = GroupKFold(n_splits=10)

    # Create a pipeline to scale data and apply Lasso
    lasso_pipeline = Pipeline([
        ('scaler', StandardScaler()),
        ('lasso_cv', LassoCV(alphas=np.logspace(-6, 0, 100), max_iter=10000, random_state=42))
    ])

    # Compute cross-validated R^2 scores
    cv_scores = cross_val_score(lasso_pipeline, X, y, groups=groups, cv=gkf, scoring='r2')
    mean_r2 = np.mean(cv_scores)
    std_r2 = np.std(cv_scores)

    return mean_r2, std_r2

# Usage example
features_base = ['Obj_Std', 'Skewness', 'Kurtosis', 'time', 'click', 'order', 'subject_id_encoded', 'Recency_Factor', 'Rep_Factor', 'Sign_Factor', 'SH_Rep_Factor', 'SH_Sign_Factor', 'SH_Obj_Std', 'SH_Skewness', 'SH_Kurtosis']
r2_8_feature_lasso, std_8_feature_lasso = lasso_group_kfold_cv(data, features_base, 'risk')
print(f"Mean R-squared for Lasso with 10-Fold CV: {r2_8_feature_lasso:.4f} ± {std_8_feature_lasso:.4f}")

Mean R-squared for Lasso with 10-Fold CV: 0.1155 ± 0.0463


In [13]:
# Extracting ret and price features
ret_features = [col for col in data.columns if col.startswith('ret_') and 'diff' not in col]
price_features = [col for col in data.columns if col.startswith('price_') and 'diff' not in col]

# Large Lasso features include base features plus all ret_ and price_ values
features_large = features_base + ret_features + price_features

# Adding first and second differences for Huge Lasso features
ret_diff_features = [col for col in data.columns if 'ret_' in col and 'diff' in col]
price_diff_features = [col for col in data.columns if 'price_' in col and 'diff' in col]

features_huge = features_large + ret_diff_features + price_diff_features

# Large Lasso
r2_large_lasso, std_r2_large_lasso = lasso_group_kfold_cv(data, features_large, 'risk')
print(f"Mean R-squared for Large Lasso with 10-Fold CV: {r2_large_lasso:.4f} ± {std_r2_large_lasso:.4f}")

# Huge Lasso
r2_huge_lasso, std_r2_huge_lasso = lasso_group_kfold_cv(data, features_huge, 'risk')
print(f"Mean R-squared for Huge Lasso with 10-Fold CV: {r2_huge_lasso:.4f} ± {std_r2_huge_lasso:.4f}")

Mean R-squared for Large Lasso with 10-Fold CV: 0.1507 ± 0.0519
Mean R-squared for Huge Lasso with 10-Fold CV: 0.1509 ± 0.0522


In [14]:
# Creating the DataFrame
dak = {
    "Model (Risk, No Subject_id)": [
        "Baseline Regression", "3-feature Regression", "8-feature Regression",
        "8-feature Lasso", "Large Lasso", "Huge Lasso", "Graph FE Regression"
    ],
    "R-squared (Without Subject ID FE)": [
        r2_base, r2_3_feature, r2_8_feature,
        r2_8_feature_lasso, r2_large_lasso, r2_large_lasso, r2_graph_id_fe
    ],
    # "R-squared (With Subject ID FE)": [
    #     r2_baseline_fe, r2_3_feature_fe, r2_8_feature_fe,
    #     r2_8_feature_lasso_fe, r2_large_lasso_fe, r2_large_lasso_fe, r2_graph_fe
    # ]
}

df = pd.DataFrame(dak)

df

,"Model (Risk, No Subject_id)",R-squared (Without Subject ID FE)
0,Baseline Regression,0.034867
1,3-feature Regression,0.109030
2,8-feature Regression,0.114209
3,8-feature Lasso,0.115514
4,Large Lasso,0.150726
5,Huge Lasso,0.150726
6,Graph FE Regression,0.169719
